In [ ]:
# from google_drive_downloader import GoogleDriveDownloader as gdd

# gdd.download_file_from_google_drive(file_id='176E-pLhoxTgWsJ3MeoJQV_GXczIA6g8D',
#                                     dest_path='/content/animals.zip',
#                                     unzip=True)
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [ ]:
IMG_SIZE = 224
class_names = [ 'elefante' , 'farfalla' ,'mucca', 'pecora' ,'scoiatt']

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255., # rescaling
                                   rotation_range = 40,  # for augmentation
                                   width_shift_range = 0.2,
                                   validation_split = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.,validation_split = 0.2)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory("train",
                                                    batch_size = 32,
                                                    subset="training",
                                                    shuffle=True,
                                                    class_mode = 'categorical', 
                                                    target_size = (224, 224))


test_generator = test_datagen.flow_from_directory("test1",
                                                    batch_size = 32,
                                                    class_mode = None, 
                                                    shuffle=False,
                                                    target_size = (224, 224))

val_generator = val_datagen.flow_from_directory("train",
                                                subset = "validation",
                                                    batch_size = 32,
                                                    class_mode = 'categorical', 
                                                    target_size = (224, 224))

Found 5113 images belonging to 5 classes.
Found 910 images belonging to 1 classes.
Found 1276 images belonging to 5 classes.


In [ ]:
INPUT_SHAPE = (224,224,3)


from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = INPUT_SHAPE, include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = True

    
#Augmenting the images
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
    ]
)
# Method 1.2: This model is same as the above in 1.1
#Sequential way of building models
model = Sequential()
model.add(data_augmentation)
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5,activation='softmax'))

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, 
                                               restore_best_weights=True,
                                               verbose=1)
model.fit(train_generator, steps_per_epoch=None, batch_size=100, shuffle=True,
          validation_data= val_generator,
          callbacks=[es_callback],
          epochs = 10)


Epoch 1/10
160/160 [==============================] - 300s 2s/step - loss: 0.8301 - accuracy: 0.7970 - val_loss: 0.3376 - val_accuracy: 0.9279
Epoch 2/10
160/160 [==============================] - 276s 2s/step - loss: 0.2899 - accuracy: 0.9269 - val_loss: 0.1649 - val_accuracy: 0.9655
Epoch 3/10
160/160 [==============================] - 273s 2s/step - loss: 0.2031 - accuracy: 0.9392 - val_loss: 0.2042 - val_accuracy: 0.9522
Epoch 4/10
160/160 [==============================] - 292s 2s/step - loss: 0.1676 - accuracy: 0.9478 - val_loss: 0.2087 - val_accuracy: 0.9647
Epoch 00004: early stopping


In [ ]:
model.save('trial_03_01.h5')

In [ ]:
import pandas as pd
import numpy as np
test_generator.reset()
preds = model.predict(test_generator)
#if using softmax activation on output layer
predicted_class_indices  = np.argmax(preds,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

df = pd.read_csv('Testing_set_animals.csv')

# results = results.set_index('Filename')
# results = results.reindex(index=df['filename'])
# results = results.reset_index()

results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

print(results.head())
results['Predictions'] = '\'\'' + results['Predictions'].astype(str) + '\''
results.sort_values(by = 'Filename', axis=0, inplace=True)
results.head()


results.to_csv('trial_003_01_02.csv')

                                   Filename       Predictions
0  test/OIP--0Yr2a_yg2hs5QgCoHAOnAHaLJ.jpeg  scoiattolo_train
1  test/OIP--2DLJjX4d5SG0eF_EXsO9AHaEk.jpeg  scoiattolo_train
2  test/OIP--2ix__438O7A-yHzROPhGwHaFj.jpeg    elefante_train
3  test/OIP-00qwuq8LDwY8bqltA0QfzwHaLF.jpeg  scoiattolo_train
4  test/OIP-01deWMs3t2fSlsMMrJhQawHaFs.jpeg      pecora_train
